<a href="https://colab.research.google.com/github/invictus125/cs598-final-project/blob/main/intraoperative_hypotension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Check git status
!cd drive/MyDrive/CS598/Final\ Project/cs598-final-project && git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   intraoperative_hypotension.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [24]:
# Copy files from git directory to working directory
!rm -rf src
!mkdir src
!cp -r drive/MyDrive/CS598/Final\ Project/cs598-final-project/src/* ./src/

In [22]:
# Copy files back to git directory from working directory
!cp -r ./src/* drive/MyDrive/CS598/Final\ Project/cs598-final-project/src/
!cd drive/MyDrive/CS598/Final\ Project/cs598-final-project && git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   intraoperative_hypotension.ipynb
	modified:   src/model.py

no changes added to commit (use "git add" and/or "git commit -a")


Link to paper: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0272055

In [20]:
# Imports
from src.model import create_1d_residual


In [21]:
# Create model building blocks
test_residual = create_1d_residual(12,3)
print(test_residual)

Linear(in_features=12, out_features=3, bias=True)
